# Prey notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes
from tqdm import tqdm

## Ideas 

* Odour - constant but noisy. 
* Sound - reliable but infrequent.
* Rather than resetting the env to zero every step, you could decay it. E.g. env[:,:,:-1] *= 0.8 + blur. 
* Analysis: n_prey caught, speed, costs (e.g. movement vs food). 
* Evolve prey against different algorithms. Then, evolve predators against these prey.   

## Rule-based Agents

In [2]:
# Hyperparameters 
size = 7
n_prey = 10
n_steps = 50
n_trials = 100
pk = 5 # the width of the prey's Gaussian signal (in rc)
scenario = "Foraging"

if scenario == "Foraging":
    pc = 0.0
    pm = None
    pe = None 
    motion = None
elif scenario == "Hunting":
    pc = 1.0
    pm = 0.8
    pe = 0.5
    motion = "Levy"

### Fitness vs noise

In [ ]:
# Fitness vs noise
noises = [0.1, 1.0, 2.0] #np.linspace(start=0.0, stop=2.0, num=10)
policies = multimodal_mazes.AgentRuleBased.policies + multimodal_mazes.AgentRuleBasedMemory.policies + ["Levy"]
colors = multimodal_mazes.AgentRuleBased.colors + multimodal_mazes.AgentRuleBasedMemory.colors + [list(np.array([24, 156, 196, 255]) / 255)]
results = np.zeros((len(noises), len(policies)))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 
        if policy in multimodal_mazes.AgentRuleBased.policies:
            agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        elif policy in multimodal_mazes.AgentRuleBasedMemory.policies:
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha = 0.6
        elif policy == "Levy":
            agnt = multimodal_mazes.AgentRandom(location=None, channels=[0,0], motion=policy)

        fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps, scenario=scenario, motion=motion, pc=pc, pm=pm, pe=pe)

        results[a, b] = fitness

for b, policy in enumerate(policies): 
    plt.plot(noises, results[:,b], color=colors[b], label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
plt.legend()

In [ ]:
# Fitness vs noise AUC 
auc = np.trapz(y=results.T, x=noises, axis=1)
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, auc[b] - auc[0])
    ml.set_color(colors[b])
    sl.set_color(colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('AUC');

### Finding alpha for the memory based agents

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=2.0, num=13)
policies = multimodal_mazes.AgentRuleBasedMemory.policies
alphas = np.linspace(start=0.0, stop=2.0, num=11)
results = np.zeros((len(noises), len(policies), len(alphas)))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 

        for c, alpha in enumerate(alphas):
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha=alpha
            fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps, scenario=scenario, pm=pm, pe=pe)
            results[a, b, c] = fitness

In [ ]:
# Fitness vs noise
colors = multimodal_mazes.AgentRuleBasedMemory.colors
auc = np.trapz(y=results.T, x=noises, axis=2)
for b, policy in enumerate(policies): 
    plt.plot(alphas, auc[:,b], color=colors[b], label=policy)

plt.ylabel('AUC')
plt.xlabel(r"$\alpha$")
plt.legend()

### Exploring task parameters 

In [ ]:
# Fitness vs noise
noises = np.linspace(start=0.0, stop=2.0, num=3)
policies = multimodal_mazes.AgentRuleBased.policies + multimodal_mazes.AgentRuleBasedMemory.policies + ["Levy"]
colors = multimodal_mazes.AgentRuleBased.colors + multimodal_mazes.AgentRuleBasedMemory.colors + [list(np.array([24, 156, 196, 255]) / 255)]
pms = np.linspace(start=0.0, stop=1.0, num=3)
pes = np.linspace(start=0.0, stop=1.0, num=3)

results = np.zeros((len(noises), len(policies), len(pms), len(pes)))

# Test agents
for a, noise in enumerate(tqdm(noises)):

    for b, policy in enumerate(policies): 
        if policy in multimodal_mazes.AgentRuleBased.policies:
            agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy=policy)
        elif policy in multimodal_mazes.AgentRuleBasedMemory.policies:
            agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy=policy)
            agnt.alpha = 0.6
        elif policy == "Levy":
            agnt = multimodal_mazes.AgentRandom(location=None, channels=[0,0], motion=policy)

        for c, pm in enumerate(pms):
            for d, pe in enumerate(pes):
                fitness, _, _, _ = multimodal_mazes.eval_predator_fitness(n_trials=n_trials, size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk, n_steps=n_steps, scenario=scenario, motion=motion, pc=pc, pm=pm, pe=pe)

                results[a, b, c, d] = fitness

# np.save("results_" + motion, results) 

#### Single experiments

In [ ]:
# Load results 
results = np.load("../results/test19/results.npy")
print(results.shape) # noises, policies, pms, pes, pcs

parameters = np.load("../results/test19/parameters.npy", allow_pickle=True)
noises = parameters.item().get("noises")
policies = parameters.item().get("policies")
pms = parameters.item().get("pms")
pes = parameters.item().get("pes")
pcs = parameters.item().get("pcs")
colors = parameters.item().get("colors")

In [ ]:
# Mean Fitness 
for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, np.mean(results[:,b]))
    ml.set_color(colors[b])
    sl.set_color(colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('Mean fitness');

In [ ]:
# Mean AUC 
auc = np.zeros((len(policies), len(pms), len(pes), len(pcs)))
for c, _ in enumerate(pms):
    for d, _ in enumerate(pes):
        for e, _ in enumerate(pcs):
                auc[:,c,d,e] = np.trapz(y=results[:,:,c,d,e].T, x=noises, axis=1)

for b, _ in enumerate(policies): 
    ml, sl, _ = plt.stem(b, np.mean(auc[b]) - np.mean(auc[0]))
    ml.set_color(colors[b])
    sl.set_color(colors[b])
plt.xticks(range(len(policies)), policies, rotation='vertical')
plt.ylabel('Normalised mean AUC');

In [ ]:
# Difference in AUC 
auc_diff = auc[-3,:,:,:] - auc[-4,:,:,:]
print(auc_diff.min(), np.argwhere(auc_diff == np.min(auc_diff))) 
print(auc_diff.max(), np.argwhere(auc_diff == np.max(auc_diff))) 

for b, policy in enumerate(policies): 
    plt.plot(noises, results[:,
        b, 
        np.argwhere(auc_diff == np.max(auc_diff))[0][0], 
        np.argwhere(auc_diff == np.max(auc_diff))[0][1],
        np.argwhere(auc_diff == np.max(auc_diff))[0][2]], 
        color=colors[b], 
        label=policy)

plt.ylim([0, 1.05])
plt.ylabel('Fitness')
plt.xlabel('Sensor Noise')
# plt.legend()

In [ ]:
# Reformat results (# noises, policies, pms, pes, pcs)
results_arrays = [[] for _ in enumerate(policies)]
for a, noise in enumerate(noises):
    for b, policy in enumerate(policies):
        for c, pm in enumerate(pms):
            for d, pe in enumerate(pes):
                for e, pc in enumerate(pcs):
                    results_arrays[b].append([noise, pm, pe, pc, results[a, b, c, d, e]])

In [ ]:
# Curve fits 
params = ["Sensor noise", "$p_m$", "$p_e$", "$p_c$"]
fig, ax = plt.subplots(nrows=1, ncols=len(params), figsize=(5*len(params),5), sharex=False, sharey=True)

for a, policy in enumerate(policies):
    data = np.array(results_arrays[a])   

    for b, param in enumerate(params):
        plt.sca(ax[b])

        # Data 
        x = data[:,b]
        y = data[:,-1]
        idx = np.argsort(x)

        # Poly fit 
        curve = np.poly1d(np.polyfit(x[idx],y[idx],deg=2))
        plt.plot(x[idx], curve(x[idx]), color=colors[a], label=policy)
        
        if (a == 0) and (b == 0): 
            plt.ylabel("Fitness")
        
        if (a==0):
            plt.xlabel(param)

plt.ylim([0, 1.05])
# plt.legend()

#### Two experiments

In [ ]:
results = np.concatenate((np.load("../results/test17/results_Brownian.npy"), np.load("../results/test17/results_Levy.npy")), axis=1)
print(results.shape)

auc = np.zeros((len(policies)*2, len(pms), len(pes)))
for c, _ in enumerate(pms):
    for d, _ in enumerate(pes):
        auc[:,c,d] = np.trapz(y=results[:,:,c,d].T, x=noises, axis=1)

for b, _ in enumerate(policies): 
    plt.scatter(np.mean(auc[b]), np.mean(auc[b + len(policies)]), color=colors[b])


## Plotting

In [ ]:
agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy='Linear fusion')
noise = 0.1
time, path, prey_state, preys = multimodal_mazes.predator_trial(size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk,n_steps=n_steps, scenario=scenario, pm=pm, pe=pe)
print(prey_state)

In [ ]:
# Plotting
from matplotlib import colors
prey_markers = ['P', 'X']

# Environment 
pk_hw = pk // 2  # half width of prey's Gaussian signal (in rc)
env = np.zeros((size, size, len(agnt.channels) + 1))
env[:, :, -1] = 1.0
env = np.pad(env, pad_width=((pk_hw, pk_hw), (pk_hw, pk_hw), (0, 0)))
plt.imshow(1 - env[:, :, -1], cmap="binary", alpha=0.25)

# Path
cmap = colors.LinearSegmentedColormap.from_list(
    "", ["xkcd:teal blue", "xkcd:off white", "xkcd:coral"], N=n_steps
)
for t in range(len(path) - 1):
    plt.plot([path[t, 1], path[t + 1, 1]], [path[t, 0], path[t + 1, 0]], c=cmap(t), zorder=0)
    plt.scatter(path[t + 1, 1], path[t + 1, 0], s=30, color=cmap(t), zorder=1)

# Prey 
for prey in preys:
    path = np.array(prey.path)
    if scenario == "Foraging":
        plt.scatter(path[0,1], path[0,0], color='k', alpha=0.5, marker=prey_markers[prey.cues], zorder=2)
    elif scenario == "Hunting":
        plt.scatter(path[-1,1], path[-1,0], color='k', alpha=0.5, marker=prey_markers[0], zorder=2)

# Adjust axes 
plt.xlim([(pk//2) - 1, size + pk//2])
plt.ylim([size + pk//2, (pk//2) - 1]) 
plt.axis("off")


### WIP: Video

In [ ]:
agnt = multimodal_mazes.AgentRuleBasedMemory(location=None, channels=[1,1], policy='Recurrent outputs')
noise = 0.1
time, path, prey_state, preys, env_log = multimodal_mazes.predator_trial(size=size, agnt=agnt, sensor_noise_scale=noise, n_prey=n_prey, pk=pk,n_steps=n_steps, scenario=scenario, motion="Levy", pc=pc, pm=pm, pe=pe, log_env=True)
print(prey_state)

In [ ]:
import matplotlib.animation as animation
prey_markers = ['P', 'X']

# Colormaps 
from matplotlib import colors
import matplotlib.cm as cm

cmap_wall = cm.binary
cmap_wall.set_under('k', alpha=0)

cmap_ch0 = colors.LinearSegmentedColormap.from_list(
    "", ["white", "xkcd:ultramarine"]
)

cmap_ch1 = colors.LinearSegmentedColormap.from_list(
    "", ["white", "xkcd:magenta"]
)

fig, ax = plt.subplots()

# Environment 
plt.imshow(1 - env_log[0][:, :, -1], clim=[0.1,1.0], cmap=cmap_wall, alpha=0.25, zorder=1)

# Adjust axes 
plt.xlim([(pk//2) - 1, size + pk//2])
plt.ylim([size + pk//2, (pk//2) - 1]) 
plt.axis("off")

# Initial data 
agnt_animation = ax.scatter([], [], s=120, color='k', zorder=3)
preys_animation = [[] for _ in preys]
for a, prey in enumerate(preys): 
    if scenario == "Foraging":
        preys_animation[a] = ax.scatter([], [], s=60, color='k', alpha=0.5, marker=prey_markers[prey.cues], zorder=2)
    elif scenario == "Hunting": 
        preys_animation[a] = ax.scatter([], [], s=60, color='k', alpha=0.5, marker=prey_markers[0], zorder=2)

# Animate 
def update_animation(t):
    plt.imshow((cmap_ch0(env_log[t][:,:,0]) + cmap_ch1(env_log[t][:,:,1]))/2, interpolation='gaussian', zorder=0) 

    agnt_animation.set_offsets([path[t, 1], path[t, 0]])

    for a, prey in enumerate(preys): 
        try:
            preys_animation[a].set_offsets([prey.path[t][1], prey.path[t][0]])
        except:
            preys_animation[a].set(alpha=0)

anim = animation.FuncAnimation(fig, update_animation, frames=range(1, len(path)), blit=False)
anim.save("Test.gif", dpi=300)


In [ ]:
# 2 channel colormap
input_values = np.linspace(0,1,num=11)
a,b = np.meshgrid(input_values, input_values)

plt.imshow((cmap_ch0(a) + cmap_ch1(b))/2, zorder=0, origin='lower')
plt.xticks(ticks=range(len(input_values)), labels=np.round(input_values,1), rotation='vertical')
plt.yticks(ticks=range(len(input_values)), labels=np.round(input_values,1))
plt.xlabel('Ch0 input')
plt.ylabel('Ch1 input')

## Evolved Agents

### Single experiments

In [18]:
path = '../Results/test20/'

In [19]:
# Load config data 
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

In [ ]:
# Load data 
x = np.load(path + '2.npy')

top_agent = np.where(x['fitness'] == x['fitness'].max())

with open(path + '2.pickle', 'rb') as file:
    genomes = pickle.load(file)

genome_id, genome, channels = genomes[top_agent[0][0]]

print(x[top_agent[0][0]])
print(genome.size())

In [ ]:
# Plotting

# Fitness 
multimodal_mazes.plot_fitness_over_generations(x, plot_species=True)

# Architecture
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D', 4 : 'Wait'}
genome = multimodal_mazes.prune_architecture(genome, config)
plt.figure()
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

### Multiple experiments

In [ ]:
# Comparing n experiments with m repeats 

# Building the feature matricies
import os
paths = ['../Results/test30/', '../Results/test31/', '../Results/test32/', '../Results/test33/', '../Results/test34/'] 

metrics_x, metrics_y, metrics_z = [], [], []
for a, path in enumerate(tqdm(paths)): 

    # Load config data 
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')
    prey_config = multimodal_mazes.load_prey_config(path + 'prey_config.ini')

    # Determine fitness cutoff 
    agnt = multimodal_mazes.AgentRuleBased(location=None, channels=[1,1], policy="Nonlinear fusion")
    fitness_cutoff, _, _, _ = multimodal_mazes.eval_predator_fitness(
                                                            n_trials=prey_config["n_trials"]*10, size=prey_config["size"], 
                                                            agnt=agnt, sensor_noise_scale=prey_config["sensor_noise_scale"],
                                                            n_prey=prey_config["n_prey"], pk=prey_config["pk"], n_steps=prey_config["n_steps"], 
                                                            scenario=prey_config["scenario"], motion=prey_config["motion"], 
                                                            pc=prey_config["pc"], pm=prey_config["pm"], pe=prey_config["pe"])
    print("Fitness_cutoff: " + str(fitness_cutoff))

    # Load data 
    for f in os.listdir(path):
        if f.endswith(".npy"):
            
            print("Loading: " + str(a) + "-" + os.path.splitext(f)[0])
            exp_data = np.load(path + f)
            with open(path + os.path.splitext(f)[0] + '.pickle', 'rb') as file:
                genomes = pickle.load(file)

            # Select top agents 
            top_agents = list(np.where(exp_data["fitness"] >= fitness_cutoff)[0])

            if top_agents:
                print("Max fitness: " + str(max(exp_data["fitness"])))

                # Calculate architecture metrics 
                top_agents_metrics_n, top_agents_metrics_p, mn_keys, mp_keys = multimodal_mazes.architecture_metrics_matrices(agents=top_agents, genomes=genomes, config=config)     
                results_x = np.concatenate((top_agents_metrics_n, top_agents_metrics_p), axis=1) 

                # Store data 
                metrics_x.append(results_x)
                metrics_y.append(np.ones(len(results_x)) * a)
                metrics_z.append(np.ones(len(results_x)) * int(os.path.splitext(f)[0]))

metrics_x = np.concatenate(metrics_x, axis=0) # agents x metrics 
metrics_y = np.concatenate(metrics_y, axis=0).astype(int) # agents, 
metrics_z = np.concatenate(metrics_z, axis=0) # agents, 
metrics_labels = list(mn_keys) + list(mp_keys) # metrics, 
metrics_type = np.concatenate((np.zeros(len(mn_keys)), np.ones(len(mp_keys)))) # metrics, 

assert len(metrics_x) == len(metrics_y) == len(metrics_z), "Mismatched data?"
assert metrics_x.shape[1] == len(metrics_labels) == len(metrics_type), "Mismatched labels?"

print(np.unique(metrics_y, return_counts=True))
print(np.unique(metrics_z, return_counts=True))

In [ ]:
from matplotlib import cm
cols = cm.get_cmap("plasma", len(np.unique(metrics_y))).colors.tolist()
offsets = np.linspace(start=-0.25, stop=0.25, num=len(np.unique(metrics_y)))

In [ ]:
# Sorting metrics 

if len(np.unique(metrics_y)) == 1:
    metrics_sorted = np.arange(start=0, stop=20)

else:
    # Comparing features 
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import cross_val_score

    model = DecisionTreeClassifier(class_weight="balanced")
    scores = cross_val_score(estimator=model, X=metrics_x, y=metrics_y, cv=10)
    print(scores)
    model.fit(metrics_x, metrics_y)
    plt.plot(model.feature_importances_) # sums to 1

    # Ranking features 
    acc_per_metric = []
    for f, _ in enumerate(metrics_labels): 
        model = DecisionTreeClassifier(class_weight="balanced", max_depth=1)
        scores = cross_val_score(estimator=model, X=metrics_x[:,f].reshape(-1,1), y=metrics_y, cv=10)
        acc_per_metric.append(scores.mean())

    # WIP: Plot ranked features 
    f, ax = plt.subplots(1, figsize=(10, 5))
    metrics_sorted = np.argsort(acc_per_metric)[::-1]
    plt.plot(np.array(acc_per_metric)[metrics_sorted], 'k')
    v, c = np.unique(metrics_y, return_counts=True)
    plt.hlines(y = max(c) / sum(c), xmin=0, xmax=len(metrics_sorted)-1, color="xkcd:gray", ls="dotted")
    plt.xticks(range(len(metrics_labels)), [metrics_labels[l] for l in metrics_sorted], rotation='vertical')
    plt.ylabel('Accuracy')

In [ ]:
# Plot just one feature
best_feature = metrics_sorted[0]
for b in np.unique(metrics_y):
    parts = plt.violinplot(dataset=metrics_x[metrics_y==b,:][:, best_feature], positions=[b], showextrema=False, showmedians=True);
    for pc in parts['bodies']:
        pc.set_facecolor(cols[b])
        pc.set_edgecolor(cols[b])
        pc.set_alpha(0.5)

    vp = parts['cmedians']
    vp.set_edgecolor(cols[b])
    vp.set_alpha(1)

plt.ylabel(metrics_labels[best_feature])
# plt.xticks([0,1], ['A', 'B'])

In [ ]:
# All features plot  
y_labels = ['Number', 'Value']

lines = []
f, ax = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.4, 0.6]}, figsize=(15, 5))
for a in [0,1]: # for each axis 
    plt.sca(ax[a])
    ms = metrics_sorted[metrics_type[metrics_sorted] == a]
    for b in np.unique(metrics_y): # for each group 
        parts = plt.violinplot(dataset=metrics_x[metrics_y==b,:][:, ms], 
                               positions=np.linspace(start=0, stop=len(ms) - 1, num=len(ms))+offsets[b], 
                               widths=0.2,
                               showextrema=False, showmedians=True);
        for pc in parts['bodies']:
            pc.set_facecolor(cols[b])
            pc.set_edgecolor(cols[b])
            pc.set_alpha(0.5)

        vp = parts['cmedians']
        vp.set_edgecolor(cols[b])
        vp.set_alpha(1)

        if a == 0: lines.append(vp)

    ax[a].set_xticks(np.arange(len(ms)), [metrics_labels[i] for i in ms], rotation='vertical')
    ax[a].set_ylabel(y_labels[a])

# Legend 
# plt.legend(lines, ['A, n=' + str(sum(metrics_y == 0)),
#                    'B, n=' + str(sum(metrics_y == 1))])

In [ ]:
for b in np.unique(metrics_y)[::-1]:
    plt.scatter(metrics_x[metrics_y==b,metrics_sorted[0]], metrics_x[metrics_y==b,metrics_sorted[1]], color=cols[b])

In [ ]:
# Embedding
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X = np.copy(metrics_x)
X = (X - np.mean(metrics_x, axis=0)) / np.std(metrics_x, axis=0) 
X_embedded = pca.fit_transform(X)
for b in np.unique(metrics_y)[::-1]:
    plt.scatter(X_embedded[metrics_y==b,0], X_embedded[metrics_y==b,1], color=cols[b])

In [ ]:
# Costs 
costs = metrics_x[:,0] + metrics_x[:,4]
 
plt.hist(costs, density=True, histtype='stepfilled', color='xkcd:grey', alpha=0.25, label='All solutions')

plt.ylabel('Density')
plt.xlabel('Cost\n($\mathregular{\eta + E}$)')
plt.legend()